In [3]:
import pytorch_lightning as pl
import sys
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from sequence_models.collaters import  MLMCollater
from pytorch_lightning.loggers import WandbLogger
from sequence_models.constants import SPECIALS
import carp_models
sys.path.append('/home/amber/multitask_RNA/rna_self_train/')
from dna_tokenizer import DNATokenizer,carp_maskwrapper
import rna_model


In [2]:
tokenizer = DNATokenizer('/home/amber/multitask_RNA/rna_self_train/vocab.txt')
decay_rate = 0.15
len_vocab = 4101
padding_idx = tokenizer.pad_token_id
data_dir = '/home/amber/multitask_RNA/data/pre-train/6mer/rna_seq.h5'
train_data = rna_model.rna_kmer(data_dir,'train',6,tokenizer)
valid_data = rna_model.rna_kmer(data_dir,'valid',6,tokenizer)
collater = carp_maskwrapper(tokenizer,decay_rate,extend = False)


In [4]:
RNA='ACGTN'
RNA_ALPHABET = RNA+SPECIALS
len_vocab = len(RNA_ALPHABET)
padding_idx = RNA_ALPHABET.index('-')
train_data = carp_models.rna_self_mask('/home/amber/multitask_RNA/data/pre-train/context/rna_seq.h5','train')
valid_data = carp_models.rna_self_mask('/home/amber/multitask_RNA/data/pre-train/context/rna_seq.h5','valid')
collater = MLMCollater(RNA_ALPHABET,True,False,mut_alphabet=RNA)


In [5]:
valid_loader = DataLoader(valid_data, num_workers=4,collate_fn = collater,batch_size = 64)

In [6]:
for i in valid_loader:
    print(i)
    break

(tensor([[3, 1, 0,  ..., 3, 7, 0],
        [2, 2, 0,  ..., 1, 1, 0],
        [2, 0, 1,  ..., 0, 2, 7],
        ...,
        [0, 2, 0,  ..., 0, 1, 1],
        [0, 2, 0,  ..., 2, 0, 1],
        [2, 1, 0,  ..., 2, 1, 3]]), tensor([[3, 1, 0,  ..., 3, 2, 0],
        [2, 2, 0,  ..., 1, 1, 0],
        [2, 0, 1,  ..., 0, 2, 3],
        ...,
        [0, 2, 0,  ..., 0, 1, 1],
        [0, 2, 0,  ..., 2, 0, 1],
        [2, 1, 0,  ..., 2, 1, 3]]), tensor([[0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]]))


In [7]:
torch.load('./protein_model/carp_76M.pt')

NameError: name 'torch' is not defined

In [13]:
RNA='ACGTN'
RNA_ALPHABET = RNA+SPECIALS

config={'model':'ByteNetLM',
                'lr':1e-3,
                'n_tokens':30,
                'd_embedding' : 8, # dimension of embedding
                'd_model': 1024, # dimension to use within ByteNet model, //2 every layer
                'n_layers' : 32, # number of layers of ByteNet block
                'activation': 'gelu',
                'kernel_size' : 5, # the kernel width
                'r' : 128, # used to calculate dilation factor
                'padding_idx' : RNA_ALPHABET.index('-') ,# location of padding token in ordered alphabet
                'dropout' : 0.1 ,
                }

model = carp_models.ByteNetLM(config['n_tokens'], config['d_embedding'], config['d_model'],
                        config['n_layers'], config['kernel_size'], config['r'], config['lr'],
                        padding_idx=config['padding_idx'],dropout=config['dropout'],final_ln=True)
model.load_state_dict(torch.load('./protein_model/carp_76M.pt')['model_state_dict'])
model.to(device)
model.eval()

ByteNetLM(
  (embedder): ByteNet(
    (embedder): Embedding(30, 8, padding_idx=6)
    (up_embedder): PositionFeedForward(
      (conv): Conv1d(8, 1024, kernel_size=(1,), stride=(1,))
    )
    (layers): ModuleList(
      (0): ByteNetBlock(
        (conv): MaskedConv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        (sequence1): Sequential(
          (0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (1): ReLU()
          (2): PositionFeedForward(
            (conv): Conv1d(1024, 512, kernel_size=(1,), stride=(1,))
          )
          (3): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (4): ReLU()
        )
        (sequence2): Sequential(
          (0): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (1): ReLU()
          (2): PositionFeedForward(
            (conv): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
          )
        )
      )
      (1): ByteNetBlock(
        (conv): MaskedConv1d(512, 512, kernel_siz